<h1>Logistische Regression mit Tensorflow anhand der MNIST Ziffern</h1>

<p>Bisher haben wir mit numpy die Modelle und deren Ableitungen manuell programmmiert. Gute Machine Learning Frameworks können uns einiges an Arbeit abnehmen. Dank automatischer Differenzierung müssen wir uns zum Beispiel keine Gedanken mehr machen, wie die Ableitung einer Funktion aussieht. Es reicht aus, wenn wir den sogenannten Vorwärtspass bis hin zur Fehlerfunktion hinschreiben und den Rest dem Framework überlassen. In den nächsten Übungen wollen wir mit der Bibliothek Tensorflow arbeiten, welche sowohl auf der CPU als auch auf der GPU läuft. Es wird empfohlen, das Framework mit Grafikkartenunterstützung zu installieren. Voraussetzung ist eine NVIDIA-Grafikkarte, die im Labor vorzufinden ist.</p>

<p>Dieses Jupyter Notebook steht wieder zum <a href="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung3/Tensorflow_Regression_MNIST_Vorlage.ipynb" target="_blank">Download</a> bereit.</p>

<hr />
<h2>Vorbereitung</h2>

<p>Die bisher verwendete numpy Bibliothek arbeitet <strong>imperative </strong>und f&uuml;hrt jede geschrieben Zeile sofort aus. Niedergeschriebene Formeln werden sofort berechnet und die Ergebnisse stehen im Anschluss zur Verf&uuml;gung. <strong>Deklarative </strong>Programmierung hingegen, beschreibt nur was berechnet werden soll, aber nicht wie und wann. Python selbst ist zwar imperativ, aber der Computation Graph von Tensorflow arbeitet deklarativ. Wir unterscheiden deshalb zwischen zwei Phasen bei der Programmierung mit Tensorflow. Zun&auml;chst erstellen wir den Graphen, das was f&uuml;r uns aussieht wie eine imerative Schreibweise ist in Wirklichkeit eine deklarative. Im zweiten Schritt erstellen wir eine Session, die den Graphen ausf&uuml;hrt und uns dessen Ergebnis ausgibt.</p>

In [ ]:
import tensorflow as tf

# work on GPU if available
with tf.device("/gpu:0"):
    
    # create a new graph and use it as the default graph
    graph = tf.Graph()
    with graph.as_default():

        # all computation operations are automatically added to the default graph
        x = tf.placeholder(tf.float32, shape=[2,3], name='x')
        y = tf.subtract(x, 5, name='y')

    # x and y are tensor operators and represent variables inside the graph. They do not hold data themself.
    # We can grab the data of a variable computed during a session and print it later.
    print(x)
    print(y)
        
    # session configuration
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    # start a new session with the new graph
    with tf.Session(graph=graph, config=config) as session:  

        # construct a subgraph to compute the values of y. Use the provided x data.
        y_ndarray = session.run(y, feed_dict={x: [[2,3,4],[8,9,8]]}) 

        # We get a numpy-array for every tensor-operation we specify in the run-function.
        print(y_ndarray)
        print(type(y_ndarray))

<hr />
<h2>Lineare Regression mit Tensorflow anhand von Beispieldaten</h2>

<p>Das folgende Beispiel erstellt zun&auml;chst zuf&auml;llige Datenpunkte und versucht dann mit Hilfe der linearen Regression eine gute approximierende Gerade zu finden. Die Daten und Vorgehensweise ist identisch zu <a href="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung1/#LinearRegression_1D_Vorlage.ipynb">&Uuml;bung 1</a>.</p>


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# enable interactive plots
%matplotlib notebook

def make_plot(x, y, prediction): 
    fig = plt.figure(figsize = (8,6))   
    
    ax = fig.add_subplot(1, 1, 1)
    ax.axis([-2, 2, 0.1, 0.6])
    ax.set_xlabel('x')
    ax.set_ylabel('y')
    
    dots, = ax.plot(x, y, 'ro')   
    line, = ax.plot(x, prediction)    
    return fig, dots, line

def show_plot(x, y, prediction): 
    fig, dots, line = make_plot(x, y, prediction)
    fig.tight_layout()
    plt.show()    

In [ ]:
np.random.seed(7) 
num_points = 100

# random data array
x = np.zeros([num_points, 1], dtype=np.float32) 
y = np.zeros([num_points, 1], dtype=np.float32)

for i in range(num_points):
    x[i] = np.random.normal(0.0, 0.55)
    y[i] = x[i] * 0.1 + 0.3 + np.random.normal(0.0, 0.03)    
    
# initiale theta-Werte
t0 =  0.4
t1 = -0.3 

# prediction
y_pred = t0 + t1*x
    
# print results
show_plot(x, y, y_pred)

<h2>Lineare Regression mit Tensorflow</h2>
<p>Weiter oben wurde kurz gezeigt, wie Tensorflow generell funktioniert. Im folgenden wurde das Beispiel erweitert und eine lineare Regression implementiert. 50 iterationen lang wird immer wieder die Kostenberechnungs- und die Gewichtsupdatefunktion aufgerufen.</p>

In [ ]:
# learn rate 
learning_rate = 0.1

# work on GPU if available
with tf.device("/gpu:0"):
    
    # computation graph
    graph = tf.Graph()
    with graph.as_default():

        # input data, infer shape at computation time
        x_input = tf.placeholder(tf.float32, name='x')
        y_input = tf.placeholder(tf.float32, name='y')

        # weights and bias initialization following Xavier Glorot and Yoshua Bengio suggestions
        xavier = tf.contrib.layers.xavier_initializer(uniform=True, seed=7, dtype=tf.float32)
        weights = tf.get_variable(shape=[1, 1], dtype=tf.float32, initializer=xavier, name='weights')
        bias = tf.get_variable(shape=[1], dtype=tf.float32, initializer=xavier, name='bias')

        # prediction (x * W + b)
        prediction = tf.add(bias, tf.matmul(x_input, weights), name='prediction')

        # mean squared error
        cost = tf.reduce_mean(tf.square(y_input - prediction), name='cost')

        # use gradient descent to derive the cost function and update the variables (weights, bias)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

    # session configuration
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    # start a new session
    with tf.Session(graph=graph, config=config) as session:  

        # initialize weights and bias variables
        session.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))     

        # cost per iteration
        predictions = []
        costs = []

        # optimise 50 iterations
        for i in range(50):
            
            # compute cost and update weights using x,y 
            predictions_out, cost_out, _ = session.run([prediction, cost, optimizer], feed_dict={x_input: x, y_input: y})
            predictions.append(predictions_out)
            costs.append(cost_out)
            
        # plot cost function
        plt.figure()
        plt.plot(costs)
        plt.show()

<h2>Ergebnis visualisieren</h2>


In [ ]:
from matplotlib.widgets import Slider

fig, dots, line = make_plot(x, y, predictions[0])
plt.subplots_adjust(left=0.08, right=0.95, top=0.95, bottom=0.15)

sliderPos = plt.axes([0.15, 0.03, 0.75, 0.03])
iterationSlider = Slider(sliderPos, 'Iteration', 0, len(predictions), valfmt="%d",valinit=0)

def update(x):
    line.set_ydata(predictions[int(x)])
    fig.canvas.draw_idle()
iterationSlider.on_changed(update)

plt.show()

<hr />

<h2>MNIST mit Logistischer Regression in Tensorflow</h2>

<p>Implementieren Sie die logistische Regression in Tensorflow, die die Ziffern der MNIST Bilder vorhersagen kann. Die Aufgabe ist identisch zu <a href="http://home.htw-berlin.de/~hezel/computervision/WS1718/uebung2/#LogisticRegression_MNIST_Vorlage.ipynb">&Uuml;bung 2</a>.</p>


In [ ]:
from shutil import copyfileobj
from sklearn.datasets.base import get_data_home
from sklearn.datasets import fetch_mldata
import os
import urllib

def fetch_mnist(data_home=None):
    mnist_alternative_url = "http://home.htw-berlin.de/~hezel/files/data/mnist-original.mat"    
    data_home = get_data_home(data_home=data_home)
    data_home = os.path.join(data_home, 'mldata')
    if not os.path.exists(data_home):
        os.makedirs(data_home)
    mnist_save_path = os.path.join(data_home, "mnist-original.mat")
    if not os.path.exists(mnist_save_path):
        print("Download MNIST to",mnist_save_path)
        mnist_url = urllib.request.urlopen(mnist_alternative_url)
        with open(mnist_save_path, "wb") as matlab_file:
            copyfileobj(mnist_url, matlab_file)
    return fetch_mldata('MNIST original')

mnist = fetch_mnist()
print(mnist)

Daten normalisieren und in Trainungs- und Testdaten einteilen.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.utils import check_random_state

# input and output data
X = mnist.data.astype('float32')
y = mnist.target

# shuffle data
random_state = check_random_state(0)
permutation = random_state.permutation(X.shape[0])
X = X[permutation]
y = y[permutation]

# split data
X_train_all, X_test_all, y_train_all, y_test_all = train_test_split(X, y, train_size=60000, test_size=10000)

# normalize
X_train = (X_train_all / 255.) 
X_test = (X_test_all / 255.)

Die y-Daten One-Hot kodieren

In [ ]:
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()

y_train_hot = enc.fit_transform(np.expand_dims(y_train_all, axis=1)).toarray()
y_test_hot = enc.fit_transform(np.expand_dims(y_test_all, axis=1)).toarray()

In [ ]:
print("number of training examples = " + str(X_train.shape[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(y_train_hot.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(y_test_hot.shape))

<hr />
<h2>Training</h2>

<p>Überführen Sie die logistischen Regression von Übung 2b in eine Tensorflow-Version. Verwenden Sie Softmax als Aktivierungsfunktion und die Cross-Entropy als Fehlerfunktion. Notieren Sie sich den Trainingsfehler anhand der Trainingsdaten und die Vorhersagegenautigkeit mit Hilfe der Testdaten. Plotten Sie den Fehler und die Genauigkeit, um zu überprüfen, ob Ihr Model funktioniert. Es werden Genauigkeiten von ungefähr 92% erwartet.</p>

In [ ]:
# TODO Build a logist regression model, using softmax and cross-entroy function

# learn rate 
learning_rate = 0.3

# work on GPU if available
with tf.device("/gpu:0"):
    
    # computation graph
    graph = tf.Graph()
    with graph.as_default():

        # input data, infer shape at computation time
        x_input = tf.placeholder(tf.float32, name='x')
        y_input = tf.placeholder(tf.float32, name='y')

        # weights and bias initialization following Xavier Glorot and Yoshua Bengio suggestions
        xavier = tf.contrib.layers.xavier_initializer(uniform=True, seed=7, dtype=tf.float32)
        weights = tf.get_variable(shape=[784, 10], dtype=tf.float32, initializer=xavier, name='weights')
        bias = tf.get_variable(shape=[1], dtype=tf.float32, initializer=xavier, name='bias')

        # prediction (x * W + b)
        z = tf.add(bias, tf.matmul(x_input, weights), name='z')

        # mean squared error
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_input, logits=z, name='cost'))
#         cost = tf.reduce_mean(tf.square(y_input - prediction), name='cost')

        # use gradient descent to derive the cost function and update the variables (weights, bias)
        optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

    # session configuration
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    # start a new session
    with tf.Session(graph=graph, config=config) as session:  

        # initialize weights and bias variables
        session.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))     

        # cost per iteration
        predictions = []
        costs = []

        # optimise 50 iterations
        for i in range(50):
            
            # compute cost and update weights using x,y 
            predictions_out, cost_out, _ = session.run([z, cost, optimizer], feed_dict={x_input: X_train, y_input: y_train_hot})
            predictions.append(predictions_out)
            costs.append(cost_out)
        
        # test prediction
        pred_test, cost_test = session.run([z, cost], feed_dict={x_input: X_test, y_input: y_test_hot})
        
        # plot cost function
        plt.figure()
        plt.plot(costs)
        plt.show()

In [ ]:
pred_train_max = (np.argmax(predictions[-1], axis=1))
y_train_max = (np.argmax(y_train_hot, axis=1))
train_acc = np.mean((pred_train_max == y_train_max))
print('Train Accuracy: {:.2f}%'.format(train_acc * 100))

pred_test_max = (np.argmax(pred_test, axis=1))
y_test_max = (np.argmax(y_test_hot, axis=1))
test_acc = np.mean((pred_test_max == y_test_max))
print('Test Accuracy: {:.2f}%'.format(test_acc * 100))

In [ ]:
_pred_test = np.array([[-1, -2, 0, 2],[-2, 1, 3, 2],[-1, 1, 0.5, 0],[1, 0.9, 0, -1.2]])
_y_test = np.array([[0, 0, 0, 1], [0, 0, 0, 1], [0, 0, 1, 0], [1, 0, 0, 0]])

print("_pred_test \n",_pred_test)
print("_y_test \n", _y_test)

print("My accuracy")
print("____________")
_y_pred_max = (_pred_test == _pred_test.max(axis=1)[:,None]).astype(int)
print("_y_pred_max\n", _y_pred_max)
diff = _y_pred_max - _y_test
print("\n_y_pred_max - _y_test\n", diff)
print("\nnp.abs(diff)\n", np.abs(diff))
print("\nnp.mean(np.abs(diff))\n", np.mean(np.abs(diff)))

print("\n\nNico accuracy")
print("____________")
_y_pred_argmax = _pred_test.argmax(axis=1)
_y_test_argmax = _y_test.argmax(axis=1)
print("\n_y_pred_argmax", _y_pred_argmax)
print("\n_y_test_argmax", _y_test_argmax)
print("==\n", _y_pred_argmax == _y_test_argmax)
print("np.mean(==)\n", np.mean(_y_pred_argmax == _y_test_argmax))


<hr />
<h2>Neural Network</h2>

<p>Laut Yann LeCun's Datenbank <a href="http://yann.lecun.com/exdb/mnist/" target="_blank">http://yann.lecun.com/exdb/mnist/</a> sind Neuronale Netzwerke mit 2-3 Schichten besser als einfache Regressionsmodelle. Erweitern Sie Ihr Model zu einem Neuronalen Netzwerk mit mindestens einem Hidden-Layer. Verwenden Sie für diese Schichten die Sigmoid- oder ReLu-Aktivierungsfunktion. Geben Sie wieder den Trainingsfehler und die Testgenauigkeit in einem Diagramm aus. Zu erwarten sind Genauigkeiten von ca. 95%.</p>

In [ ]:
# TODO Implement a neural network with at least one hidden layer. Train it and plot the error and accuracy curves.
# TODO Build a logist regression model, using softmax and cross-entroy function

# learn rate 
learning_rate = 0.01
num_hidden_1 = 256
num_hidden_2 = 64
num_outputs = 10

# work on GPU if available
with tf.device("/gpu:0"):
    
    # computation graph
    graph = tf.Graph()
    with graph.as_default():

        # input data, infer shape at computation time
        x_input = tf.placeholder(tf.float32, name='x')
        y_input = tf.placeholder(tf.float32, name='y')

        # weights and bias initialization following Xavier Glorot and Yoshua Bengio suggestions
        xavier = tf.contrib.layers.xavier_initializer(uniform=True, seed=7, dtype=tf.float32)
        weights_1 = tf.get_variable(shape=[784, num_hidden_1], dtype=tf.float32, initializer=xavier, name='weights_1')
        bias_1 = tf.get_variable(shape=[1], dtype=tf.float32, initializer=xavier, name='bias_1')

        # prediction (x * W + b)
        z_1 = tf.add(bias_1, tf.matmul(x_input, weights_1), name='z_1')
        a_1 = tf.nn.relu(z_1)
        
        weights_2 = tf.get_variable(shape=[num_hidden_1, num_outputs], dtype=tf.float32, initializer=xavier, name='weights_2')
        bias_2 = tf.get_variable(shape=[1], dtype=tf.float32, initializer=xavier, name='bias_2')
        
        # prediction (x * W + b)
        preds = tf.add(bias_2, tf.matmul(a_1, weights_2), name='z_2')
#         a_2 = tf.nn.relu(z_2)
        
#         weights_3 = tf.get_variable(shape=[num_hidden_2, num_outputs], dtype=tf.float32, initializer=xavier, name='weights_3')
#         bias_3 = tf.get_variable(shape=[1], dtype=tf.float32, initializer=xavier, name='bias_3')
        
#         # prediction (x * W + b)
#         preds = tf.add(bias_3, tf.matmul(a_2, weights_3), name='z_3')

        # mean squared error
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y_input, logits=preds, name='cost'))
#         cost = tf.reduce_mean(tf.square(y_input - prediction), name='cost')

        # use gradient descent to derive the cost function and update the variables (weights, bias)
        optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

    # session configuration
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True

    # start a new session
    with tf.Session(graph=graph, config=config) as session:  

        # initialize weights and bias variables
        session.run(tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()))     

        # cost per iteration
        predictions = []
        costs = []

        # optimise 50 iterations
        for i in range(100):
            
            # compute cost and update weights using x,y 
            predictions_out, cost_out, _ = session.run([preds, cost, optimizer], feed_dict={x_input: X_train, y_input: y_train_hot})
            predictions.append(predictions_out)
            costs.append(cost_out)
            print('Iteration [{:2}] Cost: {:.4f}'.format(str(i), cost_out))
        
        # test prediction
        pred_test, cost_test = session.run([preds, cost], feed_dict={x_input: X_test, y_input: y_test_hot})
        
        # plot cost function
        plt.figure()
        plt.plot(costs)
        plt.show()
        

pred_train_max = (np.argmax(predictions[-1], axis=1))
y_train_max = (np.argmax(y_train_hot, axis=1))
train_acc = np.mean((pred_train_max == y_train_max))
print('Train Accuracy: {:.2f}%'.format(train_acc * 100))

pred_test_max = (np.argmax(pred_test, axis=1))
y_test_max = (np.argmax(y_test_hot, axis=1))
test_acc = np.mean((pred_test_max == y_test_max))
print('Test Accuracy: {:.2f}%'.format(test_acc * 100))

<hr />

<h2>Abgabe</h2>

<p>Das von Ihnen erstellte Notebook muss sp&auml;testens bis zum 31. Dezember 2017 um 23:59 UTC+1 ;) per E-Mail an&nbsp;<a href="mailto:hezel@htw-berlin.de" target="_blank">hezel@htw-berlin.de</a>&nbsp;eingesendet werden. Verwenden Sie als Betreff bitte &quot;CV1718 &Uuml;bung3 &lt;NAME&gt;&quot; und als Notebook Name &quot;CV1718_Ue3_Tensorflow_MNIST_NAME.ipynb&quot;. Bevor Sie mir eine Mail schicken, entfernen sie bitte &uuml;ber &quot;Kernel&quot; -&gt; &quot;Restart and Clear Output&quot; s&auml;mtlichen von Python erstellten Inhalt und speichern anschlie&szlig;end das Notebook &quot;File&quot; -&gt; &quot;Save and Checkpoint&quot;.</p>
